# BulkAPI 2.0 - Update operation

## Install required libs

In [1]:
!pip install sfbulk2
!pip install Faker

## Get access token and instance url using DX
```
sfdx force:auth:web:login
```
```
Successfully authorized mohan.chinnappan.n_ea2@gmail.com with org ID 00D3h000007R1LuEAK
```
```
sfdx mohanc:hello:myorg -u mohan.chinnappan.n_ea2@gmail.com
```

```
{
    "Content-Type": "application/json",
    "Authorization": "Bearer 00D3h000007R1Lu!AR0AQLJugFTl_.Fp5HVGH39XOOvP2ldz9AG6Bbw.VggjQreqdbbmbxqhafEDlwRa0wr38CTFBhTJTXjwqCJDPMaETSFXWTZM",
    "apiVersion": "52.0",
    "instanceUrl": "https://mohansun-ea-02-dev-ed.my.salesforce.com",
    "restAPIUri": "https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v52.0/"
}
```
 

In [2]:
access_token = '00D3h000007R1Lu!AR0AQCU7eWWR7d2qJ6x6A4ZgZkLo0c6XGCrs4JmKroSB57wA8Tfjc4uispqT654xftETzR7v04t0JSWYaZ4.1Jzb0CE.pnbj'
instance_url = "https://mohansun-ea-02-dev-ed.my.salesforce.com"

In [3]:
## let import the SFBulk2 to get the sfbulk2
from sfbulk2 import SFBulk2
sfba2 = SFBulk2(access_token=access_token, instance_url=instance_url, api_version='v51.0')

api_version: v51.0


## Query the current phone nunbers for the account

In [4]:
soql = 'SELECT Id,Name, Phone from Account LIMIT 5'
query_job_id = sfba2.create_query_job(query=soql)


body: {'operation': 'query', 'query': 'SELECT Id,Name, Phone from Account LIMIT 5', 'contentType': 'CSV', 'columnDelimiter': 'COMMA', 'lineEnding': 'LF'}
uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/query
job_id: 7503h00000CV6oWAAT


In [5]:
query_results = sfba2.get_query_results(job_id=query_job_id)
query_results.text.split('\n')


['"Id","Name","Phone"',
 '"0013h00000Ffg18AAB","Boone625 Inc",""',
 '"0013h00000Ffg19AAB","Rodriquez626 Inc","881-222-2344"',
 '"0013h00000Ffg1AAAR","Hale627 Inc","882-222-2344"',
 '"0013h00000Ffg1BAAR","Stokes628 Inc","883-222-2344"',
 '"0013h00000Ffg1CAAR","Edwards629 Inc","884-222-2344"',
 '']

## Prepare the Data file for Account phone update

In [6]:

csv_url = 'https://raw.githubusercontent.com/mohan-chinnappan-n/cli-dx/master/data/accounts-update-2.csv'
import pandas as pd

In [7]:
csv = pd.read_csv(csv_url,index_col=0)

In [8]:
csv.head()

,Phone
Id,
0013h00000Ffg19AAB,881-222-2391
0013h00000Ffg1AAAR,882-222-2392
0013h00000Ffg1BAAR,883-222-2393
0013h00000Ffg1CAAR,884-222-2394


In [14]:
data = csv.to_csv()
data

'Id,Phone\n0013h00000Ffg19AAB,881-222-2391\n0013h00000Ffg1AAAR,882-222-2392\n0013h00000Ffg1BAAR,883-222-2393\n0013h00000Ffg1CAAR,884-222-2394\n'

## Create Update Job


In [9]:
update_job_id = sfba2.create_job(operation='update',obj='Account')

headers: {'Authorization': 'Bearer 00D3h000007R1Lu!AR0AQCU7eWWR7d2qJ6x6A4ZgZkLo0c6XGCrs4JmKroSB57wA8Tfjc4uispqT654xftETzR7v04t0JSWYaZ4.1Jzb0CE.pnbj', 'Content-Type': 'application/json;charset=UTF-8', 'Accept': 'application/json'}
body: {'object': 'Account', 'contentType': 'CSV', 'operation': 'update', 'lineEnding': 'LF'}
uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/ingest/
job_id: 7503h00000CV6pKAAT


## Let us get the job status and ContentUrl

In [10]:
update_res = sfba2.get_job_status(job_id=update_job_id, optype='ingest')

uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT


In [11]:
contentUrl = update_res.json()['contentUrl']

In [12]:
contentUrl

'services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT/batches'

## Put our input update data

In [15]:
put_res = sfba2.put_data(contentUrl=contentUrl,data=data)

https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT/batches


In [41]:
put_res.content

b''

## Get Update job status

In [16]:
sfba2.get_job_status(job_id=update_job_id, optype='ingest').json()

uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT


{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 51.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'contentUrl': 'services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT/batches',
 'createdById': '0053h000002xQ5sAAE',
 'createdDate': '2021-08-31T00:42:11.000+0000',
 'id': '7503h00000CV6pKAAT',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'Account',
 'operation': 'update',
 'retries': 0,
 'state': 'Open',
 'systemModstamp': '2021-08-31T00:42:11.000+0000',
 'totalProcessingTime': 0}

## Patch it to mark Upload Complete

In [17]:
sfba2.patch_state(job_id=update_job_id, state='UploadComplete').json()

uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT


{'apiVersion': 51.0,
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0053h000002xQ5sAAE',
 'createdDate': '2021-08-31T00:42:11.000+0000',
 'id': '7503h00000CV6pKAAT',
 'object': 'Account',
 'operation': 'update',
 'state': 'UploadComplete',
 'systemModstamp': '2021-08-31T00:42:11.000+0000'}

## Get Update Job Status

In [18]:
sfba2.get_job_status(job_id=update_job_id, optype='ingest').json()

uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/ingest/7503h00000CV6pKAAT


{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 86,
 'apiVersion': 51.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0053h000002xQ5sAAE',
 'createdDate': '2021-08-31T00:42:11.000+0000',
 'id': '7503h00000CV6pKAAT',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 4,
 'object': 'Account',
 'operation': 'update',
 'retries': 0,
 'state': 'JobComplete',
 'systemModstamp': '2021-08-31T00:43:11.000+0000',
 'totalProcessingTime': 207}

## Let us test to verify our update

In [19]:
soql = 'SELECT Id,Name, Phone from Account LIMIT 5'
query_job_id = sfba2.create_query_job(query=soql)
query_job_id

body: {'operation': 'query', 'query': 'SELECT Id,Name, Phone from Account LIMIT 5', 'contentType': 'CSV', 'columnDelimiter': 'COMMA', 'lineEnding': 'LF'}
uri: https://mohansun-ea-02-dev-ed.my.salesforce.com/services/data/v51.0/jobs/query
job_id: 7503h00000CV6q8AAD


'7503h00000CV6q8AAD'

In [20]:
query_results = sfba2.get_query_results(job_id=query_job_id)
query_results.text.split('\n')

['"Id","Name","Phone"',
 '"0013h00000Ffg18AAB","Boone625 Inc",""',
 '"0013h00000Ffg19AAB","Rodriquez626 Inc","881-222-2391"',
 '"0013h00000Ffg1AAAR","Hale627 Inc","882-222-2392"',
 '"0013h00000Ffg1BAAR","Stokes628 Inc","883-222-2393"',
 '"0013h00000Ffg1CAAR","Edwards629 Inc","884-222-2394"',
 '']